<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-data-into-list" data-toc-modified-id="Load-data-into-list-1">Load data into list</a></span></li><li><span><a href="#Create-graph-from-list-of-clause" data-toc-modified-id="Create-graph-from-list-of-clause-2">Create graph from list of clause</a></span></li><li><span><a href="#Find-SCC-networks-and-check-if-there-are-nodes-with-opposite-values" data-toc-modified-id="Find-SCC-networks-and-check-if-there-are-nodes-with-opposite-values-3">Find SCC networks and check if there are nodes with opposite values</a></span></li></ul></div>

Question 1
In this assignment you will implement one or more algorithms for the 2SAT problem. Here are 6 different 2SAT instances:

2sat1.txt
2sat2.txt
2sat3.txt
2sat4.txt
2sat5.txt
2sat6.txt

The file format is as follows. In each instance, the number of variables and the number of clauses is the same, and this number is specified on the first line of the file. Each subsequent line specifies a clause via its two literals, with a number denoting the variable and a "-" sign denoting logical "not". For example, the second line of the first data file is "-16808 75250", which indicates the clause -x_{16808} v x_{75250}

Your task is to determine which of the 6 instances are satisfiable, and which are unsatisfiable. In the box below, enter a 6-bit string, where the ith bit should be 1 if the ith instance is satisfiable, and 0 otherwise. For example, if you think that the first 3 instances are satisfiable and the last 3 are not, then you should enter the string 111000 in the box below.

DISCUSSION: This assignment is deliberately open-ended, and you can implement whichever 2SAT algorithm you want. For example, 2SAT reduces to computing the strongly connected components of a suitable graph (with two vertices per variable and two directed edges per clause, you should think through the details). This might be an especially attractive option for those of you who coded up an SCC algorithm in Part 2 of this specialization. Alternatively, you can use Papadimitriou's randomized local search algorithm. (The algorithm from lecture is probably too slow as stated, so you might want to make one or more simple modifications to it --- even if this means breaking the analysis given in lecture --- to ensure that it runs in a reasonable amount of time.) A third approach is via backtracking. In lecture we mentioned this approach only in passing; see Chapter 9 of the Dasgupta-Papadimitriou-Vazirani book, for example, for more details.

https://cp-algorithms.com/graph/2SAT.html


## Load data into list

In [1]:
import numpy as np
import time
import math
import sys
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from itertools import combinations 


# load data and convert to int
def read_data(filename):
    f = open(filename, 'r')
    # f.readline()
    ls = f.readline().split()
    data = []
    while ls:
        data.append([int(i) for i in ls])
        ls = f.readline().split()
    f.close()
    return data

filename = '2sat2.txt'
data = read_data(filename)
n = data[0][0]
print('number of rows: ', n)
data[:6]

number of rows:  200000


[[200000],
 [-146223, 168819],
 [-171640, 63526],
 [-51555, -198964],
 [-126068, -100807],
 [-29773, 94612]]

## Create graph from list of clause

In [2]:
def create_graph_from_2sat(data):
    """
    data is a list of list, [[1, 2], [2, -3], ...]
    each element of the list represents a clause via its two numbers, 
    with a number denoting the variable and a "-" sign denoting logical "not". 
    For example, [-16808 75250], which indicates the clause -x_{16808} v x_{75250}
    """
    graph = []  # a graph represented by a list of edges with, each edge is indicated by [start_v, end_v]
    
    for a, b in data:
        graph.append([-a, b])
        graph.append([-b, a])
        
    return graph

graph = create_graph_from_2sat(data[1:])
graph[:5]

[[146223, 168819],
 [-168819, -146223],
 [171640, 63526],
 [-63526, -171640],
 [51555, -198964]]

## Find SCC networks and check if there are nodes with opposite values 

nodes with opposite values in any scc means contradictory requirements and clauses can NOT be satisfied 

In [3]:
def convert_dict(graph):
    g = defaultdict(list)
    for e in graph:
        g[e[0]].append(e[1])
    return g

## DFS search on reversed graph
def DFS_loop(graph):
    def DFS(node):
        nonlocal t
        visited[node] = -1
        if node in graph: 
            for neighbor in graph[node]:
                if neighbor not in visited:
                    DFS(neighbor)
        t += 1
        visited[node] = t
    
    nodes = sorted(graph.keys(), reverse=True)        
    t = 0 
    global visited
    for node in nodes:
        if node not in visited:
            DFS(node)

# Graph is stored in dictionary format
def DFS_loop2(graph, nodes):
    
    def DFS(node):
        nonlocal t
        visited[node] = -1
        if node in graph: 
            for neighbor in graph[node]:
                if neighbor not in visited:
                    DFS(neighbor)
        t += 1
        visited[node] = t
        scc_nodes.append(node)
        
    scc, scc_all = [], []    
    visited = dict()
    for node in nodes:
        if node not in visited:
            scc_nodes = []
            t = 0
            DFS(node)
            scc.append(t)
            scc_all.append(scc_nodes)
            
    scc_all.sort(reverse=True, key=lambda scc: len(scc))

    return scc_all  # a list of scc, each scc is represented by a list of nodes

## DFS search on reversed graph
def DFS_loop_rev(graph):
    def DFS_C(node, t):
        nonlocal visited
        visited[node] = -1
        if node in graph: 
            for neighbor in graph[node]:
                if neighbor not in visited:
                    t = DFS_C(neighbor, t)
        t += 1
        visited[node] = t
        return t
     
    
    nodes = sorted(graph.keys(), reverse=True)        
    t = 0 
    visited = dict()
    
    for node in nodes:
        if node not in visited:
            t = DFS_C(node, t)
    
    return visited


## check if there are any 
def check_duplicate_variables(scc_all):
    for scc in scc_all:
        max_cnt = max(Counter([abs(node) for node in scc]).values())
        if max_cnt > 1:
            return True
    

## calculate scc
def find_scc(graph, i=0):
    
    global satisfiable
    # Create reverse graph g_rev in dictionary format
    g_rev = convert_dict([(e[1], e[0]) for e in graph])
    
    # run DFS on reversed graph  
    visited = DFS_loop_rev(g_rev)

    # Create graph in dictionary format
    g = convert_dict(graph)    
    
    # Run DFS on graph to find SCC
    order = sorted([[key, value] for key, value in visited.items()], key=lambda e: e[1], reverse=True)
    order = [e[0] for e in order]
    
    scc_all = DFS_loop2(g, order)
    
    print([len(scc_all[i]) for i in range(6)])
    
    satisfiable[i] = not check_duplicate_variables(scc_all)

    
    
import sys, threading

sys.setrecursionlimit(800000)
threading.stack_size(67108864)

filenames = ['2sat1.txt', '2sat2.txt', '2sat3.txt', '2sat4.txt','2sat5.txt', '2sat6.txt']
threads = []
satisfiable = [0] * len(filenames)

for i, filename in enumerate(filenames):
    data = read_data(filename)
    graph = create_graph_from_2sat(data[1:])
    thread = threading.Thread(target=find_scc, args=((graph, i)))
    threads.append(thread)
    thread.start()
    thread.join()
        
satisfiable

[6, 6, 1, 1, 1, 1]
[46, 46, 8, 4, 4, 1]
[101, 101, 47, 47, 19, 19]
[4, 4, 4, 4, 3, 3]
[38, 38, 27, 27, 15, 15]
[14, 14, 6, 4, 4, 4]


[True, False, True, True, False, False]